In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.densenet import preprocess_input

import tensorflow.lite as tflite

2023-12-19 18:53:07.141544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 18:53:07.141664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 18:53:07.154628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 18:53:07.202387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 18:53:08.097324: W tensorflow/compiler/tf2

## Loading and testing model

In [2]:
model = keras.models.load_model('./model/1702890631.h5')

2023-12-19 18:53:09.170135: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:53:09.570093: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:53:09.570143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:53:09.575267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:53:09.575322: I external/local_xla/xla/stream_executor

In [3]:
url_list = ['./img_for_test/person6_bacteria_22.jpeg', 
       './img_for_test/person19_virus_50.jpeg',
       './img_for_test/NORMAL2-IM-0030-0001.jpeg',
       './img_for_test/NORMAL2-IM-0095-0001.jpeg']

In [4]:
for elem in url_list:
    img = load_img(elem, target_size=(224, 224))
    X = np.array(img)
    X = np.array([X])
    X = preprocess_input(X)
    preds = model.predict(X)
    classes = ['normal', 'pneumonia']
    result = dict(zip(classes, preds[0]))
    print(result)

2023-12-19 18:53:26.298121: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2023-12-19 18:53:26.516900: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-19 18:53:26.581502: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 7s 7s/step
{'normal': 8.4622394e-07, 'pneumonia': 0.99999917}
1/1 [==============================] - 0s 42ms/step
{'normal': 2.4890443e-09, 'pneumonia': 1.0}
1/1 [==============================] - 0s 44ms/step
{'normal': 0.99943167, 'pneumonia': 0.00056835706}
1/1 [==============================] - 0s 42ms/step
{'normal': 0.95805526, 'pneumonia': 0.04194473}


## Convert loaded model to tflite

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('./chest_x_ray.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_zgmme4f/assets


INFO:tensorflow:Assets written to: /tmp/tmp_zgmme4f/assets
2023-12-19 18:54:59.907017: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-12-19 18:54:59.907096: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-12-19 18:54:59.908831: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_zgmme4f
2023-12-19 18:54:59.999439: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-12-19 18:54:59.999668: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_zgmme4f
2023-12-19 18:55:00.188813: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-12-19 18:55:00.333030: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-12-19 18:55:04.213265: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

## Testing tflite model

In [6]:
# Sample image for testing
url_tflite_test = './img_for_test/NORMAL2-IM-0030-0001.jpeg'
img = load_img(url_tflite_test, target_size=(224, 224))
X = np.array(img)
X = np.array([X])
X = preprocess_input(X)

In [7]:
interpreter = tflite.Interpreter(model_path='./chest_x_ray.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [8]:
interpreter.get_input_details()

[{'name': 'serving_default_input_2:0',
  'index': 0,
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'shape_signature': array([ -1, 224, 224,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [9]:
input_index = interpreter.get_input_details()[0]['index']
print(input_index)

0


In [10]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 1026,
  'shape': array([1, 2], dtype=int32),
  'shape_signature': array([-1,  2], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [11]:
output_index = interpreter.get_output_details()[0]['index']
print(output_index)

1026


In [12]:
interpreter.set_tensor(input_index, X)

In [13]:
interpreter.invoke()

In [14]:
pred = interpreter.get_tensor(output_index)
classes = ['normal', 'pneumonia']
dict(zip(classes, pred[0]))

{'normal': 0.99943167, 'pneumonia': 0.0005683562}

## Removing TF dependency

In [15]:
from PIL import Image

In [16]:
with Image.open('./img_for_test/NORMAL2-IM-0030-0001.jpeg') as img:
    img = img.convert("RGB")
    img = img.resize((224, 224), Image.NEAREST)

In [17]:
def preprocess_input(x):
    x /= 255.0
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    x[..., 0] -= mean[0]
    x[..., 1] -= mean[1]
    x[..., 2] -= mean[2]
    x[..., 0] /= std[0]
    x[..., 1] /= std[1]
    x[..., 2] /= std[2]
    return x

In [18]:
X = np.array(img, dtype='float32')
X = np.array([X])
X = preprocess_input(X)

In [19]:
interpreter.set_tensor(0, X)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)
print(pred)
classes = ['normal', 'pneumonia']
dict(zip(classes, pred[0]))

[[9.9943167e-01 5.6835619e-04]]


{'normal': 0.99943167, 'pneumonia': 0.0005683562}